In [1]:
%%writefile src/functionED.xx
#include <sycl/sycl.hpp>

void Field::evalDensity() {

    //sycl::queue q(sycl::cpu_selector_v);
    sycl::queue q(sycl::default_selector_v);
    std::cout << " Running on "
            << q.get_device().get_info<sycl::info::device::name>() << std::endl;
    
    vector<float> field;
    int natm = wf.natm;
    int npri = wf.npri;
    int norb = wf.norb;
    int npy = npoints_y;
    int npz = npoints_z;
    float x0 = xmin;
    float y0 = ymin;
    float z0 = zmin;
    float hp = delta;
    float *field_local = new float[nsize];

    std::cout << " Points ( " << npoints_x << "," << npoints_y << "," << npoints_z << ")" << std::endl;
    std::cout << " TotalPoints : " << nsize << std::endl;
    
    float *coor = new float[3 * natm];
    for (int i = 0; i < natm; i++) {
        Rvector R(wf.atoms[i].getCoors());
        coor[3 * i] = R.get_x();
        coor[3 * i + 1] = R.get_y();
        coor[3 * i + 2] = R.get_z();
    }

    {
        sycl::buffer<int, 1> icnt_buff(wf.icntrs.data(), sycl::range<1>(npri));
        sycl::buffer<int, 1> vang_buff(wf.vang.data(), sycl::range<1>(3 * npri));
        sycl::buffer<float, 1> coor_buff(coor, sycl::range<1>(3 * natm));
        sycl::buffer<float, 1> eprim_buff(wf.depris.data(), sycl::range<1>(npri));
        sycl::buffer<float, 1> coef_buff(wf.dcoefs.data(),
                                      sycl::range<1>(npri * norb));
        sycl::buffer<float, 1> nocc_buff(wf.dnoccs.data(), sycl::range<1>(norb));
        sycl::buffer<float, 1> field_buff(field_local, sycl::range<1>(nsize));
        
        q.submit([&](sycl::handler &h) {
            auto field_acc = field_buff.get_access<sycl::access::mode::write>(h);
            auto icnt_acc = icnt_buff.get_access<sycl::access::mode::read>(h);
            auto vang_acc = vang_buff.get_access<sycl::access::mode::read>(h);
            auto coor_acc = coor_buff.get_access<sycl::access::mode::read>(h);
            auto eprim_acc = eprim_buff.get_access<sycl::access::mode::read>(h);
            auto coef_acc = coef_buff.get_access<sycl::access::mode::read>(h);
            auto nocc_acc = nocc_buff.get_access<sycl::access::mode::read>(h);

// 3D index
            h.parallel_for(
            sycl::range<3>(npoints_x, npoints_y, npoints_z),
            [=](sycl::id<3> idx) {
                float cart[3];
                int k = idx[2];
                int j = idx[1];
                int i = idx[0];
                int iglob = i * npy * npz + j * npz + k;
                
                cart[0] = x0 + i * hp;
                cart[1] = y0 + j * hp;
                cart[2] = z0 + k * hp;
                
                const int *icnt_ptr =
                    icnt_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const int *vang_ptr =
                    vang_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *coor_ptr =
                    coor_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *eprim_ptr =
                    eprim_acc.get_multi_ptr<sycl::access::decorated::no>()
                        .get_raw();
                const float *nocc_ptr =
                    nocc_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                const float *coef_ptr =
                    coef_acc.get_multi_ptr<sycl::access::decorated::no>().get_raw();
                
                field_acc[iglob] =
                    Density(norb, npri, icnt_ptr, vang_ptr, cart, coor_ptr,
                                 eprim_ptr, nocc_ptr, coef_ptr);
                });
            });
        q.wait();
    }

    for (int i = 0; i < nsize; i++)
        field.push_back(field_local[i]);

#ifdef PRINTCUBE
  dumpCube(xmin, ymin, zmin, delta, npoints_x, npoints_y, npoints_z, field,
           "densitySYCL2.cube");
#endif

  delete[] coor;
  delete[] field_local;
}
